In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Embedding, Bidirectional, TimeDistributed, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive
import json

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tagged_sents = []
with open('/content/drive/MyDrive/Cleaned_Sentences_Task/cleaned_sentences.txt', 'r') as file:
    tagged_sents = json.load(file)

In [ ]:
print(tagged_sents[0])

[['आग', 'nn'], ['की', 'psp']]


In [ ]:
tagged_sentences = []
for i in tagged_sents:
    tagged_sentences.append([tuple(i[0]), tuple(i[1])])

In [ ]:
print(tagged_sentences[0])

[('आग', 'nn'), ('की', 'psp')]


In [ ]:
sentences, tags = [], []
for i in tagged_sentences:
  curr_sent, curr_tag = [], []
  for word, tag in i:
    curr_sent.append(word)
    curr_tag.append(tag.lower())
  sentences.append(curr_sent)
  tags.append(curr_tag)

In [ ]:
print(sentences[0])
print(tags[0])

['आग', 'की']
['nn', 'psp']


In [ ]:
tokenizer_sents = Tokenizer()
tokenizer_sents.fit_on_texts(sentences)
tokenized_sents = tokenizer_sents.texts_to_sequences(sentences)

In [ ]:
print(tokenized_sents[0])

[632, 3]


In [ ]:
tokenizer_tags = Tokenizer()
tokenizer_tags.fit_on_texts(tags)
tokenized_tags = tokenizer_tags.texts_to_sequences(tags)

In [ ]:
tokenized_tags[0]

[1, 2]

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer_sents.word_index) + 1, output_dim=128, input_length=2, trainable=True))
model.add(Bidirectional(LSTM(512, return_sequences=True, activation='relu')))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(256, return_sequences=True, activation='relu')))
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu')))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(128, activation='relu')))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(64, activation='relu')))
model.add(TimeDistributed(Dense(len(tokenizer_tags.word_index) + 1, activation='softmax')))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 128)            2589952   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 2, 1024)           2625536   
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 1024)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 2, 512)            2623488   
_________________________________________________________________
bidirectional_5 (Bidirection (None, 2, 256)            656384    
_________________________________________________________________
dropout_4 (Dropout)          (None, 2, 256)            0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 2, 128)           

In [ ]:
padded_sents = pad_sequences(tokenized_sents, maxlen=2, padding='post')
padded_tags = pad_sequences(tokenized_tags, maxlen=2, padding='post')

In [ ]:
model.fit(padded_sents[0: -4000], to_categorical(padded_tags[0: -4000]), batch_size=128, epochs=20, validation_split=0.2)

Epoch 1/20
2047/2047 [==============================] - 634s 310ms/step - loss: 0.6630 - acc: 0.7968 - val_loss: 0.2966 - val_acc: 0.9115
Epoch 2/20
2047/2047 [==============================] - 637s 311ms/step - loss: 0.2526 - acc: 0.9234 - val_loss: 0.2358 - val_acc: 0.9249
Epoch 3/20
2047/2047 [==============================] - 623s 305ms/step - loss: 0.2044 - acc: 0.9354 - val_loss: 0.2252 - val_acc: 0.9294
Epoch 4/20
2047/2047 [==============================] - 622s 304ms/step - loss: 0.1802 - acc: 0.9411 - val_loss: 0.2077 - val_acc: 0.9316
Epoch 5/20
2047/2047 [==============================] - 624s 305ms/step - loss: 0.1615 - acc: 0.9457 - val_loss: 0.2050 - val_acc: 0.9339
Epoch 6/20
2047/2047 [==============================] - 631s 308ms/step - loss: 0.1474 - acc: 0.9493 - val_loss: 0.2135 - val_acc: 0.9348
Epoch 7/20
2047/2047 [==============================] - 628s 307ms/step - loss: 0.1363 - acc: 0.9526 - val_loss: 0.2172 - val_acc: 0.9351
Epoch 8/20
2047/2047 [============

In [ ]:
print("Accuracy of the model =", model.evaluate(padded_sents[-4000: ], to_categorical(padded_tags[-4000: ], num_classes=len(tokenizer_tags.word_index) + 1))[1])

125/125 [==============================] - 4s 29ms/step - loss: 0.2738 - acc: 0.9352
Accuracy of the model = 0.9352499842643738


In [ ]:
actual = [list(i) for i in padded_tags[-5: ]]
actuals = []
for i in actual:
  curr = []
  for v in i:
    curr.append(int(v))
  actuals.append(curr)

In [ ]:
predictions = []
for i in model.predict_classes(padded_sents[-5: ]):
  curr = []
  for v in i:
    curr.append(int(v))
  predictions.append(curr)
print(predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
[[7, 7], [8, 10], [8, 24], [2, 5], [4, 2]]


In [ ]:
pred = tokenizer_tags.sequences_to_texts(predictions)
act = tokenizer_tags.sequences_to_texts(actuals)
print(pred)
print(act)

['prp prp', 'cc nnc', 'cc inj', 'psp jj', 'nnp psp']
['prp prp', 'cc nnc', 'cc inj', 'psp jj', 'nnp psp']


In [ ]:
predictions = model.predict_classes(padded_sents)

In [ ]:
predict = [list(i) for i in predictions]
final_predict = tokenizer_tags.sequences_to_texts(predict)

In [ ]:
rnn_tags = []
for i in final_predict:
  rnn_tags.extend(i.split(" "))

In [ ]:
print(len(tags) * 2)
print(len(rnn_tags))

662728
662728


In [ ]:
with open ('/content/drive/MyDrive/Cleaned_Sentences_Task/rnn_predictions.txt', 'w+') as file:
    json.dump(rnn_tags, file)